In [1]:
! pip install pandas
! pip install requests
! pip install plyer 


In [2]:
import pandas as pd
import requests as rq
import plyer 
import datetime
import sqlite3

In [3]:
def error(error_message):
    current_date = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    message = f'{error_message} {current_date}'
    plyer.notification.notify(
        title='Error',
        message=message,
        app_name="Alerta"
    )

In [4]:
def get_response(url):
    response = rq.get(url)
    if response.status_code == 200:
        data_json = response.json()
        return data_json
    else:
        error('Tabela nao encontrada')

In [5]:
def get_tables(url):
    response = get_response(url)
    return response

In [172]:
def create_df(data, columns):
    if data is None or not columns:
        return pd.DataFrame()

    df = pd.DataFrame(data)
    df['country'] = df['name'].apply(lambda x: x.get('common', ''))

    # Verificar se 'capital' está presente antes de acessá-lo
    if 'capital' in df.columns:
        df['capital'] = df['capital'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else x)

    df = df[columns]

    # Limpar valores nulos
    df_cleaned = df.dropna()

    return df_cleaned

In [7]:
def filter_df(df, min_population):
    return df[df['population'] > min_population]

In [8]:
def unstack_df(df, index_col):
    return df.set_index(index_col).unstack()

CRIANDO TABELA DOS PAISES

In [182]:
def create_table_countries(data, nome_tabela):
    df = pd.DataFrame(data)
    df.columns = df.columns.astype(str)
    conn = sqlite3.connect('dados_paises.db')
    cursor = conn.cursor()
    colunas_str = ', '.join([f'"{coluna}"' for coluna in df.columns])
    cursor.execute(f'CREATE TABLE IF NOT EXISTS "{nome_tabela}" ({colunas_str})')

    for index, row in df.iterrows():
        values = ', '.join([f'"{str(row[coluna])}"' for coluna in df.columns])
        cursor.execute(f'INSERT INTO "{nome_tabela}" VALUES ({values})')

    conn.commit()
    conn.close()

In [191]:
def get_countries():
    url = 'https://restcountries.com/v3.1/all?fields=name,capital,region,subregion,area,population'
    data = get_response(url)
    df = create_df(data,['country','capital','region','subregion','area','population'])
    print(df)
        
    #df_filtered = filter_df(df, 0)
    #print(f"{key} DataFrame (filtered):")
    #print(df_filtered)
        
    df_unstacked = unstack_df(df, 'country')
    print(df_unstacked)
    create_table_countries(df, 'Paises')
    

CRIANDO TABELA DOS PAISES INDEPENDENTES


In [214]:
def get_independent_countries():
    url = 'https://restcountries.com/v3.1/independent?status=true&fields=name,capital,region,subregion,area,population'
    data = get_response(url)
    df = create_df(data,['country','capital','region','subregion','area','population'])
    print(df)
        
    #df_filtered = filter_df(df, 0)
    #print(f"{key} DataFrame (filtered):")
    #print(df_filtered)
        
    df_unstacked = unstack_df(df, 'country')
    print(df_unstacked)
    create_table_countries(df, 'Independentes')
    

In [212]:
def create_table_independenty_countries(data, nome_tabela):
    # Converter a série em um DataFrame
    df = pd.DataFrame(data)

    # Converter o nome das colunas para strings
    df.columns = df.columns.astype(str)

    # Conectar ao banco de dados SQLite
    conn = sqlite3.connect('dados_paises.db')
    cursor = conn.cursor()

    # Criar a tabela no banco de dados com o nome entre aspas duplas
    colunas_str = ', '.join([f'"{coluna}"' for coluna in df.columns])
    cursor.execute(f'CREATE TABLE IF NOT EXISTS "{nome_tabela}" ({colunas_str})')

    # Inserir os dados do DataFrame na tabela
    for index, row in df.iterrows():
        values = ', '.join([f'"{str(row[coluna])}"' for coluna in df.columns])
        cursor.execute(f'INSERT INTO "{nome_tabela}" VALUES ({values})')

    # Commitar as alterações e fechar a conexão
    conn.commit()
    conn.close()

CRIANDO TABELA DAS LINGUAS


In [208]:
def create_table_languages_countries(languages_countries):
    if languages_countries:
        conn = sqlite3.connect('dados_paises.db')
        cursor = conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS Linguas
                          (Language TEXT, Country TEXT)''')
        
        for language, countries in languages_countries.items():
            for country in countries:
                cursor.execute("INSERT INTO Linguas (Language, Country) VALUES (?, ?)", (language, country))
        
        conn.commit()
        conn.close()
        print("Table 'LanguagesCountries' created and populated successfully.")
    else:
        print("Failed to create table. No data retrieved.")

In [209]:
def get_languages_countries():
    url = 'https://restcountries.com/v3.1/all?fields=name,languages'
    response = get_tables(url)
    languages_countries = {}
    for country_data in data:
        country_name = country_data.get('name', {}).get('common', '')
        languages_info = country_data.get('languages', {})
        if isinstance(languages_info, dict):  # Check if languages_info is a dictionary
            for language_code, language_info in languages_info.items():
                if isinstance(language_info, dict):  # Check if language_info is a dictionary
                    language_name = language_info.get('name', '')
                    if language_name:
                        languages_countries.setdefault(language_name, []).append(country_name)
                else:
                    languages_countries.setdefault(language_info, []).append(country_name)
        else:
            print(f"Failed to fetch data: {response.status_code}")
            return None
    create_table_languages_countries(languages_countries)

In [215]:
def main():
    get_countries()
    get_languages_countries()
    get_independent_countries()
main()

                country           capital    region       subregion  \
0               Moldova          Chișinău    Europe  Eastern Europe   
1         United States  Washington, D.C.  Americas   North America   
2               Mayotte         Mamoudzou    Africa  Eastern Africa   
3                 Nauru             Yaren   Oceania      Micronesia   
4            Mozambique            Maputo    Africa  Eastern Africa   
..                  ...               ...       ...             ...   
245               Benin        Porto-Novo    Africa  Western Africa   
246               Japan             Tokyo      Asia    Eastern Asia   
247  Dominican Republic     Santo Domingo  Americas       Caribbean   
248               Qatar              Doha      Asia    Western Asia   
249               Gabon        Libreville    Africa   Middle Africa   

          area  population  
0      33846.0     2617820  
1    9372610.0   329484123  
2        374.0      226915  
3         21.0       10834  
4 